In [1]:
import math
import pandas as pd
import random
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import cm

import mesa
from mesa.model import Model
from abm_network.model import VirusOnNetwork
from abm_network.parameters import analysis_params_a
from abm_network.constants import State
from abm_network.rule_functions import *
from abm_network.agents import MalwareAgent
from abm_network.analysis import regenerate_network, get_clusters, get_clustering_coefficient

## Run simulation

In [2]:
cluster_analysis_params = {
    "num_nodes": 10, # 100
    "avg_node_degree": 3, # 3
    "initial_outbreak_size": 1, # 1
    "centrality": "degree", # degree
    "malware_spread_chance": 0.4, # 0.4
    "malware_check_frequency":  0.4,  # 0.4[, 0.1, 0.8]
    "recovery_chance":  0.4,
    "gain_resistance_chance": gain_resistance_chance,
    "importance": importance_degree,
    "susceptible_chance":  susceptible_chance,
    "death_chance":  death_chance,
    "network": "Barabasi-Albert",
}

In [3]:
results = mesa.batch_run(
    VirusOnNetwork,
    parameters=cluster_analysis_params,
    iterations=2,
    max_steps=100,
    number_processes=1,
    data_collection_period=1,
    display_progress=True,
)

2it [00:00,  2.60it/s]


In [4]:
results

[{'RunId': 0,
  'iteration': 0,
  'Step': 0,
  'num_nodes': 10,
  'avg_node_degree': 3,
  'initial_outbreak_size': 1,
  'centrality': 'degree',
  'malware_spread_chance': 0.4,
  'malware_check_frequency': 0.4,
  'recovery_chance': 0.4,
  'gain_resistance_chance': <function abm_network.rule_functions.gain_resistance_chance(model)>,
  'importance': <function abm_network.rule_functions.importance_degree()>,
  'susceptible_chance': <function abm_network.rule_functions.susceptible_chance(model)>,
  'death_chance': <function abm_network.rule_functions.death_chance(model)>,
  'network': 'Barabasi-Albert',
  'Infected': 1,
  'Susceptible': 9,
  'Resistant': 0,
  'Offline': 0,
  'Death': 0,
  'Clusters': 1,
  'Ccoeff': 0.0,
  'Cluster_distribution': ((array([1]), array([10, 11])), (10, 10))},
 {'RunId': 0,
  'iteration': 0,
  'Step': 1,
  'num_nodes': 10,
  'avg_node_degree': 3,
  'initial_outbreak_size': 1,
  'centrality': 'degree',
  'malware_spread_chance': 0.4,
  'malware_check_frequency': 

## Analyse distribution of clusters size

In [6]:
results_df = pd.DataFrame(results)
print(results_df.keys())

Index(['RunId', 'iteration', 'Step', 'num_nodes', 'avg_node_degree',
       'initial_outbreak_size', 'centrality', 'malware_spread_chance',
       'malware_check_frequency', 'recovery_chance', 'gain_resistance_chance',
       'importance', 'susceptible_chance', 'death_chance', 'network',
       'Infected', 'Susceptible', 'Resistant', 'Offline', 'Death', 'Clusters',
       'Ccoeff', 'Cluster_distribution'],
      dtype='object')


In [7]:
results_df[(results_df.RunId == 0)]

,RunId,iteration,Step,num_nodes,avg_node_degree,initial_outbreak_size,centrality,malware_spread_chance,malware_check_frequency,recovery_chance,...,death_chance,network,Infected,Susceptible,Resistant,Offline,Death,Clusters,Ccoeff,Cluster_distribution
0,0,0,0,10,3,1,degree,0.4,0.4,0.4,...,<function death_chance at 0x1214220e0>,Barabasi-Albert,1,9,0,0,0,1,0.0,"(([1], [10, 11]), (10, 10))"
1,0,0,1,10,3,1,degree,0.4,0.4,0.4,...,<function death_chance at 0x1214220e0>,Barabasi-Albert,3,6,0,1,0,3,0.0,"(([2, 0, 0, 0, 0, 0, 0, 1], [1, 2, 3, 4, 5, 6,..."
2,0,0,2,10,3,1,degree,0.4,0.4,0.4,...,<function death_chance at 0x1214220e0>,Barabasi-Albert,3,4,1,2,0,4,0.0,"(([3, 0, 0, 0, 0, 0, 1], [1, 2, 3, 4, 5, 6, 7,..."
3,0,0,3,10,3,1,degree,0.4,0.4,0.4,...,<function death_chance at 0x1214220e0>,Barabasi-Albert,3,3,1,3,0,5,0.0,"(([4, 0, 0, 0, 0, 1], [1, 2, 3, 4, 5, 6, 7]), ..."
4,0,0,4,10,3,1,degree,0.4,0.4,0.4,...,<function death_chance at 0x1214220e0>,Barabasi-Albert,2,3,2,3,0,5,0.0,"(([4, 0, 0, 0, 0, 1], [1, 2, 3, 4, 5, 6, 7]), ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0,0,96,10,3,1,degree,0.4,0.4,0.4,...,<function death_chance at 0x1214220e0>,Barabasi-Albert,0,9,1,0,0,1,0.0,"(([1], [10, 11]), (10, 10))"
97,0,0,97,10,3,1,degree,0.4,0.4,0.4,...,<function death_chance at 0x1214220e0>,Barabasi-Albert,0,9,1,0,0,1,0.0,"(([1], [10, 11]), (10, 10))"
98,0,0,98,10,3,1,degree,0.4,0.4,0.4,...,<function death_chance at 0x1214220e0>,Barabasi-Albert,0,9,1,0,0,1,0.0,"(([1], [10, 11]), (10, 10))"
99,0,0,99,10,3,1,degree,0.4,0.4,0.4,...,<function death_chance at 0x1214220e0>,Barabasi-Albert,0,9,1,0,0,1,0.0,"(([1], [10, 11]), (10, 10))"


In [18]:
cluster_distribution = results_df[(results_df.RunId == 0)]["Cluster_distribution"]

In [113]:
test = results_df[(results_df.RunId == 0)].groupby("Infected")["Cluster_distribution"]

In [66]:
susceptible = results_df[(results_df.RunId == 0)]["Susceptible"]
infected = results_df[(results_df.RunId == 0)]["Infected"]
resistant = results_df[(results_df.RunId == 0)]["Resistant"]
offline = results_df[(results_df.RunId == 0)]["Offline"]
death = results_df[(results_df.RunId == 0)]["Death"]

In [104]:
np.unique(susceptible)
gfd = dict()
i = 1
gfd[i] = 0
gfd[i] += 1

In [156]:
k, v = np.unique(infected, return_counts=True)
dict(zip(k, v))

{0: 81, 1: 12, 2: 5, 3: 3}

In [83]:
np.repeat(np.array(susceptible).reshape((-1, 1)), 10, axis=1)

array([[9, 9, 9, ..., 9, 9, 9],
       [6, 6, 6, ..., 6, 6, 6],
       [4, 4, 4, ..., 4, 4, 4],
       ...,
       [9, 9, 9, ..., 9, 9, 9],
       [9, 9, 9, ..., 9, 9, 9],
       [9, 9, 9, ..., 9, 9, 9]])

In [45]:
def extend_hist(hist, local_range, global_range):
    nhist = np.zeros(global_range)
    nhist[local_range[0] - 1:local_range[1]] = hist
    return nhist

In [154]:
n_sim = len(cluster_distribution)
sim_cc_max = 1
sim_nhist = []
sim_avg = dict()
sim_count = dict()
avg_hist = dict()

for _, interval in cluster_distribution:
    sim_cc_max = max(sim_cc_max, cc_max)
    

for inf in np.unique(infected):
    sim_avg[inf] = np.zeros(sim_cc_max)
    sim_count[inf] = 0

for i, (tupl, interval) in enumerate(cluster_distribution):
    inf = infected[i]
    cc_min, cc_max = interval
    hist, bins = tupl
    nhist = extend_hist(hist, interval, sim_cc_max)
    sim_avg[inf] += nhist
    sim_count[inf] += 1

#     sim_nhist.append(nhist)
#     if inf in sim_avg:
#         sim_avg[inf] += nhist 
#         sim_count[inf] += 1
#     else:
#         sim_avg[inf] = nhist
#         sim_count[inf] = 0

for v in np.unique(infected): # change dict sim_avg and sim_count to get missing keys. Replace by array
    avg_hist[v] = sim_avg[v] / sim_count[v] # Generate 3d points infected-cluster_size-average_number_cluster_of_this_size

# avg_hist = sim_avg[inf]
# sim_nhist = np.array(sim_nhist)
nbins = list(range(1, sim_cc_max + 1, 1))

In [153]:
for k in sim_count.keys():
    

dict_keys([1, 3, 2, 0])

In [146]:
print(avg_hist)
print(sim_count)
print(sim_avg)
print(sim_nhist)
print(nbins)

[array([0.0125, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.0125, 1.    ]), array([1.27272727, 0.        , 0.        , 0.        , 0.        ,
       0.09090909, 0.        , 0.        , 0.90909091, 0.09090909]), array([4.  , 0.  , 0.  , 0.  , 0.  , 0.25, 1.  , 0.  , 0.  , 0.  ])]
{1: 11, 3: 2, 2: 4, 0: 80}
{1: array([14.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 10.,  1.]), 3: array([9., 0., 0., 0., 0., 1., 1., 1., 0., 0.]), 2: array([16.,  0.,  0.,  0.,  0.,  1.,  4.,  0.,  0.,  0.]), 0: array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., 80.])}
[[14.  0.  0. ...  0. 10.  1.]
 [ 9.  0.  0. ...  1.  0.  0.]
 [ 3.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  1.]
 [ 0.  0.  0. ...  0.  0.  1.]
 [ 0.  0.  0. ...  0.  0.  1.]]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [60]:
# Ne fait pas sens. 
# On ne veut pas la moyenne sur l'ensemble de la simulation.
# On veut comparer par rapport au nombre de S,I,R,O,D ou autre parametre
# Potentiellement detecter des avalanches
np.sum(sim_nhist, axis=0) / n_sim

array([0.3960396 , 0.        , 0.        , 0.        , 0.        ,
       0.02970297, 0.04950495, 0.00990099, 0.10891089, 0.8019802 ])

## Analyse number of avalanche compare to S,I,R,O,D

In [ ]:
nb_clusters = results_df[(results_df.RunId == 0)]["Cluster_distribution"]